In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
from utils.helper_functions import * 

dirnames = ["Asia", "Europe", "North_America" ]
ref_sets = [["Maharashtra", "India", "Asia", "Global"], ["Netherlands", "Europe", "Global"],["Connecticut", "USA", "North_America", "Global"]]

allele_freqs = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
ref_sets = ref_sets
abundances = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]


produce result files

In [ ]:

# for dir, ref in zip(dirnames, ref_sets):
#     # print(dir, ref)
#     output_results_to_json_3_dirs(dir,"0.1", ref, allele_freqs, abundances, "B.1.1.7_sequence")

In [ ]:

# for dir, ref in zip(dirnames, ref_sets):
#     # print(dir, ref)
#     output_results_to_json_3_dirs(dir,"0.1", ref, allele_freqs, abundances, "B.1.1.7_sequence", "VOC", "alpha")

read results

In [ ]:
results = dict()

for continent in dirnames: 
    results[continent] = dict()
    # load results
    with open("results_{}.json".format(continent)) as json_file:
        results[continent] = json.loads(json_file.read())


In [ ]:
results_who = dict()

for continent in dirnames: 
    results_who[continent] = dict()
    # load results
    with open("results_who_{}.json".format(continent)) as json_file:
        results_who[continent] = json.loads(json_file.read())

calculate absolute errors

In [ ]:
ref_sets_dict = dict()

for continent, ref_set_list in zip(dirnames, ref_sets):
    ref_sets_dict[continent] = ref_set_list

# (results_dict, third_dirs, abundances, second_dirs, first_dirs
absolute_errors = calculate_absolute_errors_af(results, allele_freqs, abundances, ref_sets_dict, dirnames, True)

In [ ]:

absolute_errors_who = calculate_absolute_errors_af(results_who, allele_freqs, abundances, ref_sets_dict, dirnames, True)

In [ ]:
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np

def plot_with_line_plots_two_scales_custom(continents, ref_sets_dict, absolute_errors, outdir, allele_freqs):
    fig = plt.figure(figsize=(20, 10))
    fig.set_dpi(100)
    outer = gridspec.GridSpec(1, 1, wspace=0.2, hspace=0.2)
    colors = ["orange", "green", "red", "orange", "green", "red"]
    for continent , i in zip(continents, range(3)):
        if continent != "North_America":
            continue
        
        if continent == "North_America":
            i = 0
        inner = gridspec.GridSpecFromSubplotSpec(1, 2,
                                subplot_spec=outer[i], wspace=0.1, hspace=0.1)
        axParent = plt.Subplot(fig, outer[i])
        # axParent.set_title("{}".format(continent.replace("_", " ")), fontweight='bold', pad=20, fontsize = 20)
        axParent.set_xlabel("Allele frequency threshold",  labelpad=-40, fontsize = 20 )
        axParent.set_xticks([])
        axParent.set_yticks([])
        axParent.spines["top"].set_visible(False)
        axParent.spines["bottom"].set_visible(False)
        axParent.spines["left"].set_visible(False)
        axParent.spines["right"].set_visible(False)
        fig.add_subplot(axParent)
        
        for j, scale in zip(range(2), [[0, 10], [9, 20]]):
                ax = plt.Subplot(fig, inner[j])
                if j == 0:
                    range_txt = "one to ten"
                else: 
                    range_txt = "ten to hundred"

                ax.set_title("Abundances in the range of {}".format(range_txt), pad=15, fontsize = 20)
                fig.add_subplot(ax)
                for reference_set in ref_sets_dict[continent]:
                    sums = []
                    for af in allele_freqs:
                        af = str(af)
                        sum_per_af = sum(list(absolute_errors[continent][reference_set][af].values())[scale[0]: scale[1]])/10
                        sums.append(sum_per_af)
                    
                    if continent == "Europe":
                        print("in, ", reference_set)
                        ax.plot(allele_freqs, sums, label = reference_set.replace("_", " "), color = colors[0])
                        colors.pop(0)
                    else: 
                        ax.plot(allele_freqs, sums, label = reference_set.replace("_", " "))  

           
                    ax.set_box_aspect(16/27)         
                    ax.set_yticks((0, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100))
                    ax.set_xticks(allele_freqs)
                    ax.set_ylabel("Average absolute error", fontsize = 20, labelpad = 20)
                    ax.set_ylim(0,15)
                    ax.tick_params(axis='both', labelsize=20)
                    ax.label_outer()
                

                    # put legend outside of plot and have it to be common for all plots
                    if j == 1:
                        ax.legend(bbox_to_anchor=(1.05, 0.7), loc='upper left', borderaxespad=0., fontsize = 20)

    

    # save absolute error as pdf in figures folder
    plt.savefig(outdir + "/lineplots_custom.pdf", bbox_inches='tight')


In [ ]:
from utils.plotting_functions import *
if os.path.isdir("figures") != True:
    os.mkdir("figures")


# plot_with_line_plots_two_scales(dirnames, ref_sets_dict, absolute_errors, "figures", allele_freqs)

if os.path.isdir("figures_who") != True:
    os.mkdir("figures_who")
    
# plot_with_line_plots_two_scales(dirnames, ref_sets_dict, absolute_errors_who, "figures_who", allele_freqs)
plot_with_line_plots_two_scales_custom(dirnames, ref_sets_dict, absolute_errors_who, "figures_who", allele_freqs)